In [1]:
import numpy as np
import re

In [2]:
mask = 'X'*36

In [26]:
def convert_num_to_binary(num):
    return f'{int(bin(num)[2:]):036d}'

def apply_mask(binary, mask):
    num = list(binary)
    mask_list = list(mask)
    new_binary = ''.join([num[i] 
                          if mask_list[i] == 'X'
                          else mask_list[i] 
                          for i in range(36)
                         ])
    return int(new_binary, 2)


In [31]:
mem_address = {}

with open('input_day14.txt', 'r') as f:
    for line in f.readlines():
        line=line.strip()
        
        if line.startswith('mask'):
            mask = line.split(' ')[2]
        else:
            m = re.search('mem\[(\d+)\] = (\d+)', line)
            mem = m.groups()[0]
            binary = convert_num_to_binary(int(m.groups()[1]))
            mem_address[mem] = apply_mask(binary, mask)
            

In [33]:
sum(mem_address.values())

10035335144067

In [62]:
import itertools

def get_all_combos(a_list):
    all_combinations = []
    for r in range(len(a_list) + 1):
    
        combinations_object = itertools.combinations(a_list, r)
        combinations_list = list(combinations_object)
        all_combinations += combinations_list
    return all_combinations

def apply_mask_mem(mem_num, mask):
    mask_list = list(mask)
    locs_xs = [ i for i,val in enumerate(mask_list) if val == 'X']
    
    mem_bin = convert_num_to_binary(int(mem_num))
    
    mem_bin = ''.join([ '1' if mask_list[i] == '1' else val for i,val in enumerate(list(mem_bin)) ])
   
    locations=[]
    for combo in get_all_combos(locs_xs):
        new_bin = [ val 
                    if i not in locs_xs
                    else  '0'  if i in combo
                    else '1'
                    for i,val in enumerate(mem_bin)
                  ]
       
        new_bin = ''.join(new_bin)
        locations.append(int(new_bin,2))
    return locations
        
    

In [66]:
mem_address_pt2 = {}

with open('input_day14.txt', 'r') as f:
    for line in f.readlines():
        line=line.strip()
        
        if line.startswith('mask'):
            mask = line.split(' ')[2]
        else:
            m = re.search('mem\[(\d+)\] = (\d+)', line)
            mem = m.groups()[0]
            num = m.groups()[1]
            for locs in apply_mask_mem(mem, mask):
                mem_address_pt2[str(locs)] = int(num)
            

In [67]:
sum(mem_address_pt2.values())

3817372618036

# Day 15

In [72]:
start_numbers = [0,13,16,17,1,10,6]
#start_numbers = [3,1,2]

In [77]:
i=len(start_numbers)+1
last_val = str(start_numbers[-1])

number_dict = {str(val):i+1 for i,val in enumerate(start_numbers)}

while i < 30000001:
    if last_val in number_dict.keys():
        old_iter = number_dict[str(last_val)]
        number_dict[str(last_val)] = i - 1
        last_val = str(i - old_iter-1)
    else:
        number_dict[str(last_val)] = i-1
        last_val = '0'
    i+=1
    
        


In [78]:
last_val

'31916'

In [76]:
len(number_dict)

375

# Day 16

In [26]:
import re

In [7]:
rules = {}
with open('input_day16.txt', 'r') as f:
    for line in f.readlines():
        line=line.strip()
        if len(line)==0:
            break
        field_str = line.split(':')[0]
        val_str = line.split(':')[1].split(' or ')
        val_str = [ [ int(j) for j in i.split('-')] for i in val_str]
        
        rules[field_str] = val_str
        
        

In [16]:
def check_rules(value, rules):
    for field, limits in rules.items():
        for limit in limits:
            if  limit[0] <= value <= limit[1]:
                return True
    return False 
            

In [24]:
bad_vals=[]
with open('input_day16.txt', 'r') as f:
    read_ind = False
    for line in f.readlines():
        if line.startswith('nearby tickets'):
            read_ind =True
            continue
            
        if read_ind:
            values =line.strip().split(',')
            
            cur_vals = [ int(i) for i in values if not check_rules(int(i), rules)]
            bad_vals += cur_vals
        

In [25]:
sum(bad_vals)

27802

In [27]:
tickets = []

with open('input_day16.txt', 'r') as f:
    for line in f.readlines():
        m = re.search(',', line)
        if m is None:
            continue
            
        values =[ int(i) for i in line.strip().split(',') ]
        cur_vals = [ i for i in values if not check_rules(i, rules)]
        if len(cur_vals)  == 0:
            tickets.append(values)
        

In [30]:
def get_fields(value, rules):
    applicable_fields = []
    for field, limits in rules.items():
        for limit in limits:
            if  limit[0] <= value <= limit[1]:
                applicable_fields.append(field)
                
    return set(applicable_fields)
            

In [31]:
possible_fields = [ get_fields(i, rules) for i in tickets[0]]

In [34]:
for ticket in tickets[1:]:
    possible_fields = [possible_fields[i].intersection(get_fields(value, rules))
                       for i,value in enumerate(ticket)
                      ]

In [45]:
len(possible_fields)

20

In [84]:
known_locs = {}
cnt = 0
while len(known_locs) < 20 and cnt < 30:
    
    for field in set(rules.keys()) - set(known_locs.keys()):
        truth_array = [ field in (fset - set(known_locs.keys())) for fset in possible_fields]
        
        if sum(truth_array) == 1:
            known_locs[field] = truth_array.index(True)
        
    for i,(features, poss_len) in enumerate([ (fset - set(known_locs.keys()), 
                                               len(fset - set(known_locs.keys())) ) for fset in possible_fields]):
        if poss_len == 1:
            if list(features)[0] not in known_locs.keys():
                known_locs[list(features)[0]]= i
    cnt+=1
    

In [88]:
known_locs

{'row': 3,
 'arrival station': 12,
 'train': 4,
 'duration': 15,
 'route': 7,
 'wagon': 0,
 'zone': 2,
 'class': 19,
 'arrival track': 18,
 'arrival platform': 10,
 'departure time': 8,
 'departure date': 17,
 'departure platform': 14,
 'departure track': 11,
 'departure location': 1,
 'departure station': 6,
 'price': 9,
 'seat': 16,
 'type': 5,
 'arrival location': 3}

In [89]:
my_ticket = tickets[0]

In [92]:
multiple_val = 1
for i in [ my_ticket[known_locs[field]] for field in known_locs.keys() if field.startswith('departure')]:
    multiple_val *= i

In [93]:
multiple_val

279139880759

# Day 17

In [56]:
import numpy as np

In [57]:
def check_neighbors(loc, grid):
    count_active = 0
    status = grid[loc]
    for i in range(-1,2):
        for j in range(-1,2):
            for k in range(-1,2):
                if i==j==k==0 :
                    continue
                count_active += grid[loc[0]+i, loc[1]+j, loc[2]+k ]
    if status ==1:
        if 1 < count_active < 4:
            return 1
        else:
            return 0
    if status == 0:
        if count_active == 3:
            return 1
        else:
            return 0

In [58]:
init_grid = np.zeros((30,30,20))

In [59]:
with open('input_day17.txt', 'r') as f:
    for i,line in enumerate(f.readlines()):
        line = line.strip()
        elements = list(line)
        for j,sym in  enumerate(elements):
            if sym=='#':
                init_grid[10+i, 10+j, 10] = 1


In [60]:
init_grid[10:19,10:19,10]

array([[0., 1., 0., 1., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 1., 0.],
       [1., 1., 1., 1., 1., 0., 0., 1., 0.],
       [1., 1., 1., 1., 1., 0., 0., 1., 0.],
       [1., 1., 1., 1., 1., 0., 0., 1., 0.],
       [1., 1., 1., 0., 0., 1., 0., 1., 0.],
       [1., 0., 0., 1., 1., 0., 1., 1., 0.],
       [1., 0., 1., 0., 1., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [61]:
for step in range(6):
    next_grid = init_grid.copy()
    for x in range(1,29):
        for y in range(1,29):
            for z in range(1,19):
                next_grid[x,y,z] = check_neighbors((x,y,z), init_grid)
    init_grid = next_grid.copy()

In [62]:
init_grid.sum()

375.0

In [64]:
def check_neighbors_4d(loc, grid):
    count_active = 0
    status = grid[loc]
    for i in range(-1,2):
        for j in range(-1,2):
            for k in range(-1,2):
                for l in range(-1,2):
                    if i==j==k==l==0 :
                        continue
                    count_active += grid[loc[0]+i, loc[1]+j, loc[2]+k ,loc[3]+l ]
    if status ==1:
        if 1 < count_active < 4:
            return 1
        else:
            return 0
    if status == 0:
        if count_active == 3:
            return 1
        else:
            return 0

In [65]:
init_grid = np.zeros((30,30,16,16))

In [66]:
with open('input_day17.txt', 'r') as f:
    for i,line in enumerate(f.readlines()):
        line = line.strip()
        elements = list(line)
        for j,sym in  enumerate(elements):
            if sym=='#':
                init_grid[10+i, 10+j, 8,8] = 1


In [67]:
init_grid[10:19,10:19,8,8]

array([[0., 1., 0., 1., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 1., 0.],
       [1., 1., 1., 1., 1., 0., 0., 1., 0.],
       [1., 1., 1., 1., 1., 0., 0., 1., 0.],
       [1., 1., 1., 1., 1., 0., 0., 1., 0.],
       [1., 1., 1., 0., 0., 1., 0., 1., 0.],
       [1., 0., 0., 1., 1., 0., 1., 1., 0.],
       [1., 0., 1., 0., 1., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [68]:
for step in range(6):
    next_grid = init_grid.copy()
    for x in range(1,29):
        for y in range(1,29):
            for z in range(1,15):
                for w in range(1,15):
                    next_grid[x,y,z,w] = check_neighbors_4d((x,y,z,w), init_grid)
    init_grid = next_grid.copy()

In [69]:
init_grid.sum()

2192.0

# Day 18

In [32]:
def add_ints(x,y):
    return int(x)+int(y)

def multi_ints(x,y):
    return int(x)*int(y)

ops_func = {'+': add_ints,
       '*': multi_ints,
      }

In [1]:
test_line = '((5 + 7 * 2 * 9) + (2 + 3 * 6 + 2 * 5 * 8) * 8 * (8 + 8 * 6) + 5 * (7 * 9)) + 7 * 9 * (6 + 8)'

In [25]:
test_line = test_line.replace(' ','')

In [62]:
def evaluate_line(line):
    line = '(' + line + ')'
    ops = []
    vals =[]
    
    i=0
    while i <len(line):
        if line[i] in ('(', '+', '*'):
            ops.append(line[i])
        
        elif line[i].isdigit() :
            j=i
            while line[i:j+1].isdigit() :
                j+=1
            vals.append(int(line[i:j]))
            i=j-1
        
        elif line[i] == ')':
            tmp_ops = []
            tmp_vals = []
            
            tmp_ops.append(ops.pop())
            tmp_vals.append(vals.pop())
            tmp_vals.append(vals.pop())
            
            old_op = ops.pop()
            
            while old_op != '(':
                tmp_vals.append(vals.pop())
                tmp_ops.append(old_op)
                old_op = ops.pop()
                
            while len(tmp_ops) != 0:
                val1 = tmp_vals.pop()
                val2 = tmp_vals.pop()
                oper = tmp_ops.pop()
                tmp_vals.append(ops_func[oper](val1,val2))
            vals.append(tmp_vals.pop())
        i+=1
        
    if len(ops) > 0:
        ops.reverse()
        vals.reverse()
        
        while len(ops) != 0:
            val1 = vals.pop()
            val2 = vals.pop()
            oper = ops.pop()
            vals.append(ops_func[oper](val1,val2))
    return vals[0]
        

In [63]:
evaluate_line('8+3')

11

In [64]:
evaluate_line(test_line)

9120231036

In [65]:
evaluate_line('1 + (2 * 3) + (4 * (5 + 6))'.replace(' ',''))

51

In [66]:
total_sum=0
with open('input_day18.txt', 'r') as f:
    for line in f.readlines():
        line = line.strip().replace(' ','')
        total_sum += evaluate_line(line)


In [67]:
total_sum

5783053349377

In [71]:
tmp =['*', '*', '+', '*','+']
tmp2 = [1,2,3,4,5,6]

In [72]:
idx = tmp.index('+')

In [74]:
tmp

['*', '*', '*', '+']

In [95]:
def use_precedance(opers, vals):
    
    while '+' in opers:
        idx = opers.index('+')
        val1 = vals[idx]
        val2 = vals[idx+1]
        
        vals[idx] = val1+val2
        vals.pop(idx+1)
        opers.pop(idx)
        
    mult=1
    for val in vals:
        mult*=val
    return mult
    
    
def evaluate_line_add(line):
    line = '(' + line + ')'
    ops = []
    vals =[]
    
    i=0
    while i <len(line):
        if line[i] in ('(', '+', '*'):
            ops.append(line[i])
        
        elif line[i].isdigit() :
            j=i
            while line[i:j+1].isdigit() :
                j+=1
            vals.append(int(line[i:j]))
            i=j-1
        
        elif line[i] == ')':
            tmp_ops = []
            tmp_vals = []
            
            tmp_ops.append(ops.pop())
            tmp_vals.append(vals.pop())
            tmp_vals.append(vals.pop())
            
            old_op = ops.pop()
            
            while old_op != '(':
                tmp_vals.append(vals.pop())
                tmp_ops.append(old_op)
                old_op = ops.pop()
                
            vals.append(use_precedance(tmp_ops, tmp_vals))
        i+=1
    return use_precedance(ops,vals)
        

In [96]:
evaluate_line_add('1 + 2 * 3 + 4 * 5 + 6')

231

In [97]:
evaluate_line_add('1 + (2 * 3) + (4 * (5 + 6))')

51

In [98]:
evaluate_line_add('5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))')

669060

In [99]:
total_sum=0
with open('input_day18.txt', 'r') as f:
    for line in f.readlines():
        line = line.strip()
        total_sum += evaluate_line_add(line)


In [100]:
total_sum

74821486966872

# Playground

In [36]:
test_case =(np.random.rand(100000) < 0.6).astype(int)

In [37]:
def find_max_sum_balanced(arr):
    total_ones = sum(arr)
    total_size = len(arr)
    
    max_size = min(total_ones, total_size-total_ones)*2
    
    for test_size in range(max_size,0,-2):
        start_idx =0
        diff =0
        set_ones = sum(arr[start_idx:test_size])
        
        if set_ones == test_size//2:
            return test_size, arr[start_idx:start_idx+test_size]
            
        while start_idx < total_size-test_size:
            diff = np.abs(total_size//2 - set_ones)
            end_idx =start_idx+test_size
            if end_idx + diff > total_size:
                break
                
            set_ones += sum(arr[end_idx:end_idx+diff]) - sum(arr[start_idx:start_idx+diff])
            start_idx += diff
            if set_ones == test_size//2:
                return test_size, arr[start_idx:start_idx+test_size]
            
    return 0, []
            
        
        

In [38]:
find_max_sum_balanced(test_case)

KeyboardInterrupt: 